# Classic Genetic Algorithm

In [8]:
import os
import csv
import random
import datetime
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Experimental settings

In [ ]:
MAX_LENGTH     = 200          # 유전자 최대 길이
POP_SIZE       = 1000         # 인구 크기
PARENTS_RATIO  = 0.1          # 상위 몇 %를 부모(엘리트)로 선택할지
GENERATIONS    = 500          # 최대 세대 수
SEQ_LENGTH     = 200          # 초기 시퀀스 길이
SAVE_LOG_PATH  = "./logs"     # 로그를 저장할 폴더
EARLY_STOP_THRESHOLD = 1e-3   # 조기 종료 조건: fitness가 이 값 미만이면 중단
CROSSOVER_PORTION = {
    "single_point": 0.25,
    "two_point":    0.25,
    "uniform":      0.25,
    "block_based":  0.25
}  # 교차 적용 비율 (합 1)
MUTATION_PROBS = {
    "point":            0.2,
    "swap":             0.2,
    "inversion":        0.2,
    "shuffle":          0.2,
    "segment_rotation": 0.1,
    "insert_inverse":   0.05,
    "remove_inverse":   0.05
}  # 교차 적용 비율 (합 1 이하)

TARGET_UNITARY = np.array([[0, -1],
                           [1,  0]], dtype=complex)

## Utilities

### Helper functions

In [10]:
ALLOWED_ANYON = [0, 1, 2, 3]      # Fibonacci anyon 4종 (identity 제외)
INVERSE_MAP   = {0: 2, 1: 3, 2: 0, 3: 1}  # anyon와 그 역원

def pad_to_equal_length(seq1, seq2):
    """
    두 시퀀스의 길이가 다르면, 짧은 시퀀스에 랜덤 위치에 identity (-1)를 삽입하여
    두 시퀀스의 길이를 같게 만드는 함수.
    """
    parent1 = list(seq1)
    parent2 = list(seq2)
    len1, len2 = len(parent1), len(parent2)
    if len1 < len2:
        while len(parent1) < len2:
            pos = random.randint(0, len(parent1))
            parent1.insert(pos, -1)
    elif len2 < len1:
        while len(parent2) < len1:
            pos = random.randint(0, len(parent2))
            parent2.insert(pos, -1)
    return parent1, parent2

def remove_identity(seq):
    """시퀀스에서 identity (-1)를 모두 제거합니다."""
    return [gene for gene in seq if gene != -1]

def reduce_word(seq):
    """
    주어진 시퀀스에서, allowed anyon (0,1,2,3) 간의 인접한 역원 쌍을 
    stack 방식을 통해 취소합니다.
    """
    INVERSE_MAP = {0: 2, 1: 3, 2: 0, 3: 1}
    stack = []
    for gene in seq:
        if stack and gene == INVERSE_MAP.get(stack[-1]):
            stack.pop()
        else:
            stack.append(gene)
    return stack

### Crossover operators

In [11]:
def single_point_crossover(seq1, seq2):
    """
    Single-point crossover.
    
    두 부모 시퀀스의 길이가 다르면 짧은 쪽을 identity(-1)로 패딩하여 길이를 맞춘 뒤,
    단일 교차점을 기준으로 교차한 후, offspring에서 identity를 제거하여 반환합니다.
    """
    parent1, parent2 = pad_to_equal_length(seq1, seq2)
    n = len(parent1)
    if n < 2:
        return remove_identity(parent1), remove_identity(parent2)
    
    point = random.randint(1, n - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    
    return remove_identity(child1), remove_identity(child2)

def two_point_crossover(seq1, seq2):
    """
    Two-point crossover.
    
    두 부모 시퀀스의 길이를 먼저 맞춘 뒤, 두 교차점을 선택하여 중간 구간을 서로 교환합니다.
    최종 offspring에서 identity (-1)는 제거됩니다.
    """
    parent1, parent2 = pad_to_equal_length(seq1, seq2)
    n = len(parent1)
    if n < 3:
        # 길이가 너무 짧으면 single-point로 대체
        return single_point_crossover(seq1, seq2)
    
    pt1 = random.randint(1, n - 2)
    pt2 = random.randint(pt1 + 1, n - 1)
    child1 = parent1[:pt1] + parent2[pt1:pt2] + parent1[pt2:]
    child2 = parent2[:pt1] + parent1[pt1:pt2] + parent2[pt2:]
    
    return remove_identity(child1), remove_identity(child2)

def uniform_crossover(seq1, seq2):
    """
    Uniform crossover.
    
    두 부모 시퀀스의 길이를 먼저 맞춘 뒤, 각 위치별로 50% 확률로 부모의 유전자를 선택하여
    두 offspring을 생성합니다. 마지막에 identity는 제거됩니다.
    """
    parent1, parent2 = pad_to_equal_length(seq1, seq2)
    n = len(parent1)
    child1, child2 = [], []
    for i in range(n):
        if random.random() < 0.5:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])
    return remove_identity(child1), remove_identity(child2)

def block_based_crossover(seq1, seq2, block_size=5):
    """
    Block-based crossover.
    
    두 부모 시퀀스의 길이를 맞춘 뒤, 길이가 block_size 이상이면
    임의의 블록을 선택하여 해당 블록을 교환합니다.
    길이가 block_size보다 작으면 single-point crossover로 대체합니다.
    최종 offspring에서 identity는 제거됩니다.
    """
    parent1, parent2 = pad_to_equal_length(seq1, seq2)
    n = len(parent1)
    if n <= block_size:
        return single_point_crossover(seq1, seq2)
    
    start = random.randint(0, n - block_size)
    end = start + block_size
    child1 = parent1[:start] + parent2[start:end] + parent1[end:]
    child2 = parent2[:start] + parent1[start:end] + parent2[end:]
    
    return remove_identity(child1), remove_identity(child2)

def crossover(seq1, seq2):
    """
    두 부모 시퀀스에 대해, CROSSOVER_PORTION에 명시된 비율에 따라
    'single_point', 'two_point', 'uniform', 'block_based' 교차 방법 중 하나를 랜덤으로 선택하여 적용합니다.
    
    (만약 두 시퀀스의 길이가 다르면, 내부적으로 짧은 쪽에 identity (‑1)를 삽입해 길이를 맞춘 후 교차하고,
    최종 offspring에서는 identity를 제거합니다.)
    
    반환값은 두 자식 시퀀스의 튜플입니다.
    """
    method = random.choices(
        list(CROSSOVER_PORTION.keys()),
        weights=list(CROSSOVER_PORTION.values()),
        k=1
    )[0]
    
    if method == "single_point":
        offspring = single_point_crossover(seq1, seq2)
    elif method == "two_point":
        offspring = two_point_crossover(seq1, seq2)
    elif method == "uniform":
        offspring = uniform_crossover(seq1, seq2)
    elif method == "block_based":
        offspring = block_based_crossover(seq1, seq2)
    else:
        offspring = (seq1.copy(), seq2.copy())
    
    return offspring

### Mutation operators

In [12]:
def point_mutation(seq):
    """
    각 위치의 유전자를 1/len(seq) 확률로 임의의 allowed anyon으로 대체합니다.
    """
    new_seq = list(seq)
    if len(new_seq) == 0:
        return new_seq
    rate = 1.0 / len(new_seq)
    for i in range(len(new_seq)):
        if random.random() < rate:
            new_seq[i] = random.choice(ALLOWED_ANYON)
    return new_seq

def swap_mutation(seq):
    """
    두 위치의 유전자를 무조건 swap 합니다.
    """
    new_seq = list(seq)
    if len(new_seq) < 2:
        return new_seq
    i, j = random.sample(range(len(new_seq)), 2)
    new_seq[i], new_seq[j] = new_seq[j], new_seq[i]
    return new_seq

def inversion_mutation(seq):
    """
    선택된 구간의 유전자 순서를 뒤집습니다.
    """
    new_seq = list(seq)
    if len(new_seq) < 2:
        return new_seq
    i, j = sorted(random.sample(range(len(new_seq)), 2))
    new_seq[i:j] = new_seq[i:j][::-1]
    return new_seq

def shuffle_mutation(seq):
    """
    선택된 구간 내의 유전자 순서를 무작위로 섞습니다.
    """
    new_seq = list(seq)
    if len(new_seq) < 2:
        return new_seq
    i, j = sorted(random.sample(range(len(new_seq)), 2))
    sub = new_seq[i:j]
    random.shuffle(sub)
    new_seq[i:j] = sub
    return new_seq

def segment_rotation_mutation(seq):
    """
    선택된 구간을 임의의 오프셋만큼 회전시킵니다.
    """
    new_seq = list(seq)
    if len(new_seq) < 2:
        return new_seq
    i, j = sorted(random.sample(range(len(new_seq)), 2))
    segment = new_seq[i:j]
    if len(segment) > 1:
        k = random.randint(1, len(segment)-1)
        new_seq[i:j] = segment[k:] + segment[:k]
    return new_seq

def insert_inverse_mutation(seq):
    """
    시퀀스 내 임의의 부분 구간 앞뒤에 임의의 anyon와 그 역원 쌍을 삽입합니다.
    예: [X1, ..., Xi, ..., Xj, ..., Xn] → [X1, ..., A, Xi, ..., Xj, A⁻¹, ..., Xn]
    """
    new_seq = list(seq)
    if len(new_seq) == 0:
        A = random.choice(ALLOWED_ANYON)
        new_seq.extend([A, INVERSE_MAP[A]])
        return new_seq
    i = random.randint(0, len(new_seq)-1)
    j = random.randint(i, len(new_seq)-1)
    A = random.choice(ALLOWED_ANYON)
    A_inv = INVERSE_MAP[A]
    new_seq.insert(i, A)
    new_seq.insert(j+1, A_inv)
    return new_seq

def remove_inversion_mutation(seq):
    """
    시퀀스 내에서 anyon와 그 역원 쌍이 인접(혹은 떨어져 있음)하면,
    그 중 하나의 쌍을 찾아 제거합니다.
    """
    new_seq = list(seq)
    candidate_pairs = []
    n = len(new_seq)
    for i in range(n-1):
        A = new_seq[i]
        if A not in ALLOWED_ANYON:
            continue
        for j in range(i+1, n):
            if new_seq[j] == INVERSE_MAP.get(A):
                candidate_pairs.append((i, j))
    if candidate_pairs:
        i, j = random.choice(candidate_pairs)
        # 인덱스 j부터 삭제하여 i의 인덱스 영향을 피함
        del new_seq[j]
        del new_seq[i]
    return new_seq

def mutation(seq):
    """
    MUTATION_PROBS에 따라 확률적으로 mutation을 적용합니다.
    만약 선택된 mutation 방법이 있다면 해당 함수를 적용하고,
    적용하지 않을 확률의 경우 원래 시퀀스를 그대로 반환합니다.
    최종적으로, free-group reduction을 통해 인접한 역행렬 쌍을 취소합니다.
    """
    r = random.random()
    cum_prob = 0.0
    selected_method = None
    for method, prob in MUTATION_PROBS.items():
        cum_prob += prob
        if r < cum_prob:
            selected_method = method
            break
    if selected_method is None:
        # 아무 mutation도 적용하지 않음
        new_seq = list(seq)
    else:
        if selected_method == "point":
            new_seq = point_mutation(seq)
        elif selected_method == "swap":
            new_seq = swap_mutation(seq)
        elif selected_method == "inversion":
            new_seq = inversion_mutation(seq)
        elif selected_method == "shuffle":
            new_seq = shuffle_mutation(seq)
        elif selected_method == "segment_rotation":
            new_seq = segment_rotation_mutation(seq)
        elif selected_method == "insert_inverse":
            new_seq = insert_inverse_mutation(seq)
        elif selected_method == "remove_inverse":
            new_seq = remove_inversion_mutation(seq)
        else:
            new_seq = list(seq)
    # free-group reduction을 적용하여 인접한 역행렬 취소
    new_seq = reduce_word(new_seq)
    return new_seq

## Fitness function

In [13]:
# 필요한 상수/행렬 정의 (함수 밖에서 전역적으로 선언)
phi = (1 + np.sqrt(5)) / 2

sigma_1 = np.array([
    [np.exp(-4j * np.pi / 5), 0],
    [0, np.exp(3j * np.pi / 5)]
])
sigma_2 = np.array([
    [np.exp(4j * np.pi / 5) / phi,         np.exp(-3j * np.pi / 5) / np.sqrt(phi)],
    [np.exp(-3j * np.pi / 5) / np.sqrt(phi), -1 / phi]
])
sigma_1_inv = np.linalg.inv(sigma_1)
sigma_2_inv = np.linalg.inv(sigma_2)

# seq에서 사용할 인덱스 -> 행렬 매핑
generators_map = {
    0: sigma_1,
    1: sigma_2,
    2: sigma_1_inv,
    3: sigma_2_inv
}

possible_indices = [0, 1, 2, 3]


def fitness(seq, unitary):
    """
    Calculate the Frobenius distance between the final matrix 
    (obtained by applying the generator sequence to 'unitary') 
    and the phase-adjusted identity.

    Parameters
    ----------
    seq : list of int
        A list of indices in {0,1,2,3}, where:
            0 -> sigma_1
            1 -> sigma_2
            2 -> sigma_1_inv
            3 -> sigma_2_inv
    unitary : np.array
        The target 2x2 unitary we multiply from the left.

    Returns
    -------
    distance : float
        The Frobenius norm distance to the phase-adjusted identity.
    """
    # unitary를 복사한 뒤, seq에 있는 각 인덱스에 대응하는 행렬을 곱해준다.
    M = np.array(unitary, copy=True)
    for idx in seq:
        gate = generators_map[idx]  # 0->sigma_1, 1->sigma_2, 등
        M = np.dot(M, gate)

    # 최종 M의 트레이스를 통해 전역 위상(phase)을 계산
    trace_M = np.trace(M)
    if abs(trace_M) > 1e-12:
        phase = trace_M / abs(trace_M)
    else:
        phase = 1.0

    # phase를 곱한 identity로부터의 Frobenius norm 거리
    optimal_matrix = phase * np.eye(2, dtype=complex)
    distance = np.linalg.norm(M - optimal_matrix, 'fro')
    return distance

## Genetic algorithm

### Target Unitary

In [14]:
def genetic_algorithm(unitary):
    """
    주어진 2x2 유니터리 행렬 unitary를 근사하기 위해 GA를 실행합니다.
    전역변수로 정의된 파라미터를 사용합니다.
    """

    # 로그 폴더 생성
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    log_dir = os.path.join(SAVE_LOG_PATH, timestamp)
    os.makedirs(log_dir, exist_ok=True)
    
    # CSV 로그 초기화
    csv_file_path = os.path.join(log_dir, "log.csv")
    with open(csv_file_path, "w", newline="", encoding="utf-8") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Generation", "Best Fitness", "Best Sequence"])

    # 초기 population 생성: 각 개체는 possible_indices에서 무작위 선택하여 SEQ_LENGTH 길이의 리스트로 생성
    population = [
        [random.choice(possible_indices) for _ in range(SEQ_LENGTH)]
        for _ in range(POP_SIZE)
    ]

    best_fitness_history = []
    best_seq_global = None
    best_fitness_global = float('inf')

    print("=== Genetic Algorithm Start ===")
    print(f"Population Size: {POP_SIZE}")
    print(f"Parents Ratio: {PARENTS_RATIO}")
    print(f"Generations: {GENERATIONS}")
    print(f"Sequence Length: {SEQ_LENGTH}")
    print(f"Mutation Rate: {MUTATION_PROBS}\n")

    # 메인 GA 루프
    with tqdm(range(GENERATIONS)) as pbar:
        for generation in pbar:
            # 각 개체의 fitness 평가
            # fitness(seq, unitary) 함수는 내부적으로 index -> 행렬 매핑 (generators_map) 을 사용함
            fitness_scores = [(seq, fitness(seq, unitary)) for seq in population]
            fitness_scores.sort(key=lambda x: x[1])  # 낮은 fitness가 좋은 경우
            best_seq, best_fit = fitness_scores[0]

            # 전역 best 업데이트
            if best_fit < best_fitness_global:
                best_fitness_global = best_fit
                best_seq_global = best_seq

            best_fitness_history.append(best_fit)

            # 엘리트 선택: 상위 PARENTS_RATIO 비율만큼 선택
            num_elites = max(1, int(POP_SIZE * PARENTS_RATIO))
            elites = [seq for seq, _ in fitness_scores[:num_elites]]
            new_population = elites.copy()

            # 새로운 개체 생성 (crossover 후 mutation 적용)
            while len(new_population) < POP_SIZE:
                parent1 = random.choice(elites)
                parent2 = random.choice(elites)
                child1, child2 = crossover(parent1, parent2)
                child1 = mutation(child1)
                child2 = mutation(child2)
                new_population.extend([child1, child2])
            population = new_population[:POP_SIZE]

            pbar.set_description(f"Generation {generation+1}")
            pbar.set_postfix(best_fitness=best_fit)

            with open(csv_file_path, "a", newline="", encoding="utf-8") as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow([generation+1, best_fit, best_seq])

            if best_fit < EARLY_STOP_THRESHOLD:
                print("Early stopping condition met!")
                break

    print("\n=== Genetic Algorithm Finished ===")
    print(f"Best Fitness: {best_fitness_global}")
    print(f"Best Sequence: {best_seq_global}\n")
    print(f"Length of the Best Sequence: {len(best_seq_global)}")

    # Fitness 변화 곡선 플롯 및 저장
    plt.figure(figsize=(8, 5))
    plt.plot(best_fitness_history, label="Best Fitness")
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.title("Fitness over Generations")
    plt.legend()
    plt.grid(True)
    plot_path = os.path.join(log_dir, "fitness_plot.png")
    plt.savefig(plot_path, dpi=150)
    plt.close()

    return best_seq_global, best_fitness_global, log_dir

best_seq, best_fit, log_dir = genetic_algorithm(TARGET_UNITARY)
print(f"Results saved in: {log_dir}")


=== Genetic Algorithm Start ===
Population Size: 1000
Parents Ratio: 0.2
Generations: 500
Sequence Length: 200
Mutation Rate: {'point': 0.2, 'swap': 0.2, 'inversion': 0.2, 'shuffle': 0.2, 'segment_rotation': 0.1, 'insert_inverse': 0.05, 'remove_inverse': 0.05}



Generation 500: 100%|██████████| 500/500 [00:40<00:00, 12.46it/s, best_fitness=0.0205]



=== Genetic Algorithm Finished ===
Best Fitness: 0.02045526679832879
Best Sequence: [2, 3, 3, 2, 2, 3, 0, 1, 2, 2, 3, 2, 3, 0, 3, 2, 2, 3, 3, 0, 1, 2, 3, 0, 1, 1, 2, 2, 3, 3, 0, 0, 1, 0, 0, 3, 0, 0, 0, 1, 1, 2, 3, 3, 3, 2, 1, 2, 3, 0, 0, 0, 1, 2, 1, 1, 2, 2, 1, 0, 3, 0, 3, 2, 1, 2, 3, 2]

Length of the Best Sequence: 68
Results saved in: ./logs\20250213_012740
